# Linux Sysadmin DeCal Grade Processor
Last updated: Spring 2021

Created by bencuan

## About
This tool use Gradescope grade exports and generates an interactive grade report. (TODO: Automatically email students with their grade)

There is a precommit hook that automatically removes all output from the notebook, so don't worry about accidentally exposing sensitive data. Just don't commit the csv's themselves.

## Install
To run Jupyter notebooks, you can [install it here](https://jupyter.org/install) or use [JupyterHub](https://datahub.berkeley.edu/). vscode also seems to have ipynb support now so that's pretty cool. If running locally, ensure you have pandas (`pip install pandas`).

## Usage
1. Export grade CSV's from Gradescope and place them in the `data/` folder.
1. In the code box below, assign `beginner_path` and `advanced_path` to point to the respective csv's.
2. If the lab names changed, you may also need to modify the `beginner_labs` and `advanced_labs` variables. Make them a list with each entry being a string containing one lab name (e.g. 'b1')
4. Run all of the cells!
5. 
5. The final cell exports the result to the file `data/grades_compiled.csv`.

In [3]:
# IMPORTS
import pandas as pd

In [4]:
# CONFIGURATION
beginner_path = 'beginner.csv'
advanced_path = 'advanced.csv'

beginner_labs = [f'b{i}' for i in range(1, 11)] + ['11']
advanced_labs = [f'a{i}' for i in range(1, 11)] + ['11']

In [5]:
print('hi')


hi


SyntaxError: invalid syntax (<ipython-input-6-4b37ef281455>, line 1)

## Export Grades for Submission
The script below takes in an empty grade sheet (contact Hilfinger or Cindy to get) and fills it in with P's and NP's. It is recommended that you manually look over the output and enter comments for all of the NP's.

All of the above cells must be run before this section.

In [ ]:
# CONFIGURATION
ONTIME_LAB_MIN = 7 # At least this number of labs turned in on time for P
COMPLETE_LAB_MIN = 10 # At least this number of labs turned in for P